In [ ]:
with open('/content/the-verdict.txt','r', encoding="utf-8") as f:
    raw_text = f.read()

In [ ]:
print(len(raw_text))

20479


Now we want to tokenize this 20479 character that we can later turn into embeddings to LLMs

In [ ]:
# now we will use regular expression Python library to split any given text
import re

processed = re.split(r'([,.;":?!_()\']|--|\s)', raw_text)
processed = [item.strip() for item in processed if item.strip()]

In [ ]:
print(processed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [ ]:
print(len(processed))

4690


Converting Tokens into token IDs

In [ ]:
# now we give token to any id
all_words = sorted(set(processed))
print(len(all_words))

1130


In [ ]:
# now we assign each word to a unique number
vocab = {token:integer for token,integer in enumerate(all_words)}

In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}


    def encode(self,text):
        preprocessed = re.split(r'([,.;:"()\'] |--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids


    def decoder(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])',r'\1',text)
        return text


In [ ]:
tokenizer = SimpleTokenizerV1(vocab)


In [ ]:
all_tokens = sorted(list(set(processed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [ ]:
# @title
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:t for t,i in vocab.items()}


    def encode(self,text):
        preprocessed = re.split(r'([,.;:?!"()\]|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        processed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decoder(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\']+)',r'\1',text)
        return text

### Byte pair encoding


In [ ]:
import tiktoken

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
text = "Hey, this is dkfjs;irfpiodskf"

In [ ]:
ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

In [ ]:
string = tokenizer.decode(ids)

In [ ]:
string


'Hey, this is dkfjs;irfpiodskf'

### input target pairs


In [ ]:
with open('/content/the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

In [ ]:
len(raw_text)

20479

In [ ]:
enc_text = tokenizer.encode(raw_text)
enc_text[:9]

[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899]

In [ ]:
enc_sample = enc_text[:50]

In [ ]:
context_size = 4

for i in range(0, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    # print(context, '-->', desired)
    # print(tokenizer.decode(context), '-->', tokenizer.decode([desired]))
    print(context)

[]
[40]
[40, 367]
[40, 367, 2885]
[40, 367, 2885, 1464]


In [ ]:
import torch

In [ ]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self,tokenizer, text, context_size, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        for i in range(0,len(text)-context_size,stride):
            self.input_ids.append(torch.tensor(token_ids[i:i+context_size]))
            self.target_ids.append(torch.tensor(token_ids[i+1 : i+1+context_size]))

    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]





In [ ]:
def create_dataloader_v1(text, context_size=256, stride=128, batch_size=4, shuffle=True, drop_last=True, num_workers=0):

    tokenizer = tiktoken.get_encoding("gpt2")

    #create dataset
    dataset = GPTDatasetV1(tokenizer, text, context_size, stride)

    #create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [ ]:
my_data_loader = create_dataloader_v1(raw_text, 4, 1, 1)

In [ ]:
abc = iter(my_data_loader)

In [ ]:
print(next(abc))

[tensor([], size=(1, 0)), tensor([], size=(1, 0))]


In [ ]:
second_batch = next(abc)
print(second_batch)

[tensor([[ 5574,  1660,    12, 49903]]), tensor([[ 1660,    12, 49903,   438]])]


In [ ]:
### token embeddings example


### token embeddings example

In [ ]:
# the token embedding vector has size of vocab size(no. of tokens) X output dimention

In [ ]:
token_length = 5
features = 3

import torch

torch.manual_seed(123)

embedding_layer = torch.nn.Embedding(token_length, features)

In [ ]:
embedding_layer.weight # this is random at first

Parameter containing:
tensor([[-0.1115,  0.1204, -0.3696],
        [-0.2404, -1.1969,  0.2093],
        [-0.9724, -0.7550,  0.3239],
        [-0.1085,  0.2103, -0.3908],
        [ 0.2350,  0.6653,  0.3528]], requires_grad=True)

In [ ]:
# it is lookup operation.

## Positional Embedding


In [ ]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
max_len = 4
dataloader = create_dataloader_v1(raw_text, max_len, max_len, 8)
data_iter = iter(dataloader)
input, target = next(data_iter)

In [ ]:
token_embeddings = token_embedding_layer(input)


In [ ]:
context_len = max_len
pos_embedding_layer = torch.nn.Embedding(context_len, output_dim)

In [ ]:
pos_embeddings = pos_embedding_layer(torch.arange(max_len))

In [ ]:
input_embeddings = pos_embeddings + token_embeddings

## Self Attention Mechanism

In [ ]:
# we have token embedding scores.
# we first initialize 0 attention scores for each token.
# take token_embedding score for query
# do dot product of that query with each token embedding -> attention score


In [ ]:
import torch

In [ ]:
# token_embedding scores
inputs = torch.tensor([
   [0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55] # step     (x^6)
])

In [ ]:
query = inputs[1] # our query
attn_scores = torch.empty(inputs.shape[0]) # our attn_score
print(attn_scores)


tensor([-9.0889e+15,  7.0368e+28,  2.8183e+20,  5.4058e-14,  9.8612e+18,
         2.9264e+29])


In [ ]:
for i,vector in enumerate(inputs):
    # print(i,'---',vector)
    attn_scores[i] = torch.dot(vector, query)


In [ ]:
print(attn_scores)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [ ]:
# now we normalize it. with softmax
norm_attn_scores = torch.softmax(attn_scores, dim=0)

In [ ]:
print(torch.sum(norm_attn_scores))

tensor(1.)


In [ ]:
context_vector_x2 = torch.zeros(query.shape)
context_vector_x2

tensor([0., 0., 0.])

In [ ]:
# to get embedding vector here for x2
# multiply each attn_score with corresponding token_embedding vector we got and sum them

for i, token_embedding_vector in enumerate(inputs):
    context_vector_x2 += norm_attn_scores[i] * token_embedding_vector


In [ ]:
context_vector_x2


tensor([0.4419, 0.6515, 0.5683])

In [ ]:
# now, we will do this for each token with respect to each token.
# find each attention_weight for each different token, taking it as a query.
# this can be done as input.input transpose

In [ ]:
attn_scores = inputs @ inputs.T

In [ ]:
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [ ]:
attn_weights = torch.softmax(attn_scores, dim=-1)

In [ ]:
all_context_vecs = attn_weights @ inputs

In [ ]:
all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

## Self Attention with trainable weights

In [ ]:
import torch

In [ ]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [ ]:
# here, we have 3 trainable weight metrices. key, query, weight.
# we gain them by matrix multiplication with input and their correspondig weight metrics

In [ ]:
x2 = inputs[1] #finding context vec of x2 with respect to other vectors
x2

tensor([0.5500, 0.8700, 0.6600])

In [ ]:
d_in = inputs.shape[1]
d_out = 2

In [ ]:
# defining weight for key, query, value
torch.manual_seed(123)
Wq = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
Wk = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
Wv = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [ ]:
print(Wq)

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])


In [ ]:
# now, we will multiply token_embedding vector with this weights to get weight, key, query metrics

In [ ]:
keys = inputs @ Wk

In [ ]:
keys

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]])

In [ ]:
values = inputs @ Wv
queries = inputs @ Wq

In [ ]:
# now, to get attn_scores, we do matrix multiplication of queries and key transpose

In [ ]:
attn_scoress = queries @ keys.T

In [ ]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scoress / d_k**0.5, dim=-1)

In [ ]:
context_vec_2 = attn_weights_2 @ values

In [ ]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec

In [ ]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

## Causal Attention

In [ ]:
torch.manual_seed(123)
sa_v2 = SelfAttention_v2(d_in, d_out)

In [ ]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)


In [ ]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[0.1717, 0.1762, 0.1761, 0.1555, 0.1627, 0.1579],
        [0.1636, 0.1749, 0.1746, 0.1612, 0.1605, 0.1652],
        [0.1637, 0.1749, 0.1746, 0.1611, 0.1606, 0.1651],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.1632, 0.1674],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.1639],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)


In [ ]:
# causal attention hides attention weights above diagonal. and we take softmax after that.
# but, in above, we already have influence of future tokens.


In [ ]:
# so we do different way.
# we use tril function of python torch to create mask

In [ ]:
context_length = 6

In [ ]:
torch.ones(context_length, context_length)

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

In [ ]:
mask_simple = torch.tril(torch.ones(context_length, context_length))
mask_simple

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])

In [ ]:
# now applying this mask to our attn_weights
updated_attn_wts = attn_weights * mask_simple
updated_attn_wts

tensor([[0.1717, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1636, 0.1749, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1637, 0.1749, 0.1746, 0.0000, 0.0000, 0.0000],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.0000, 0.0000],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<MulBackward0>)

In [ ]:
row_sum = updated_attn_wts.sum(dim=1, keepdim=True)
row_sum

tensor([[1.0000],
        [1.6667],
        [2.2121],
        [2.6921],
        [3.1301],
        [3.5382]])

In [ ]:
updated_attn_wts = mask_simple / row_sum

In [ ]:
updated_attn_wts

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6000, 0.6000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4521, 0.4521, 0.4521, 0.0000, 0.0000, 0.0000],
        [0.3715, 0.3715, 0.3715, 0.3715, 0.0000, 0.0000],
        [0.3195, 0.3195, 0.3195, 0.3195, 0.3195, 0.0000],
        [0.2826, 0.2826, 0.2826, 0.2826, 0.2826, 0.2826]])

In [ ]:
# this is so fucking weird, let's just do better way

In [ ]:
# what we do is, we create mask with '1' value in upper tringular, replace it with -inf value.
# then take softmax, so it turns out to be 0.

In [ ]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
mask

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])

In [ ]:
attn_scores # not normalized.

tensor([[0.3111, 0.3479, 0.3471, 0.1714, 0.2350, 0.1928],
        [0.1655, 0.2602, 0.2576, 0.1445, 0.1384, 0.1790],
        [0.1667, 0.2602, 0.2577, 0.1443, 0.1391, 0.1784],
        [0.0510, 0.1080, 0.1064, 0.0643, 0.0476, 0.0835],
        [0.1415, 0.1875, 0.1863, 0.0987, 0.1121, 0.1174],
        [0.0476, 0.1192, 0.1171, 0.0731, 0.0477, 0.0966]],
       grad_fn=<MmBackward0>)

In [ ]:
masked_attn_scores = attn_scores.masked_fill(mask.bool(), float('-inf'))
masked_attn_scores

tensor([[0.3111,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.1655, 0.2602,   -inf,   -inf,   -inf,   -inf],
        [0.1667, 0.2602, 0.2577,   -inf,   -inf,   -inf],
        [0.0510, 0.1080, 0.1064, 0.0643,   -inf,   -inf],
        [0.1415, 0.1875, 0.1863, 0.0987, 0.1121,   -inf],
        [0.0476, 0.1192, 0.1171, 0.0731, 0.0477, 0.0966]],
       grad_fn=<MaskedFillBackward0>)

In [ ]:
# now we do softmax
attn_weights = torch.softmax(masked_attn_scores / keys.shape[-1]** 0.5, dim=-1)
attn_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)

In [ ]:
## Dropout example
example = torch.ones(6,6)
# example
dropout = torch.nn.Dropout(0.2)
print(dropout(example))

tensor([[2., 0., 2., 2., 2., 2.],
        [2., 0., 2., 0., 2., 2.],
        [0., 0., 2., 0., 2., 0.],
        [2., 0., 2., 0., 2., 2.],
        [2., 2., 0., 2., 0., 0.],
        [0., 0., 0., 0., 0., 2.]])


In [ ]:
dropout(attn_weights)

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.9665, 1.0335, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6380, 0.0000, 0.6804, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5090, 0.5085, 0.4936, 0.0000, 0.0000],
        [0.3988, 0.4120, 0.4116, 0.0000, 0.0000, 0.0000],
        [0.3249, 0.3418, 0.0000, 0.3308, 0.3249, 0.0000]],
       grad_fn=<MulBackward0>)

In [ ]:
# implementing class for this.
batch = torch.stack((inputs, inputs), dim=0)
batch

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])

In [ ]:
# 1) initializing query, key, value
# 2) find attention score
# 3) masking them
# 4) find weights
# 5) find context vector

In [ ]:
import torch.nn as nn

In [ ]:
class Causal_attention(nn.Module):

    def __init__(self, d_in, d_out, context_length, qkv_bias=False, dropout=0.2):
        super().__init__()

        self.W_queries = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_keys = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_values = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))


    def forward(self, x):
        no_of_batches, token_nos, d_in = x.shape
        keys = self.W_keys(x)
        queries = self.W_queries(x)
        values = self.W_values(x)
        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:token_nos, :token_nos], -torch.inf)  # Changed num_tokens to token_nos
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values
        return context_vec

## Implementing Multi head attention

In [ ]:
class MultiheadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, num_heads, context_length, qkv_bias=False, dropout=0.2):
        super().__init__()
        self.heads = nn.ModuleList(
            [ Causal_attention(d_in, d_out, context_length, qkv_bias, dropout)
            for _ in range(num_heads)]
        )

    def forward(self, x):
         return torch.cat([head(x) for head in self.heads], dim=-1)

In [ ]:
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [ ]:
torch.manual_seed(123)
context_length = batch.shape[1] # This is the number of tokens = 6
d_in, d_out = 3, 2
mha = MultiheadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

In [ ]:
mha1 = MultiheadAttentionWrapper(d_in=inputs.shape[-1], d_out=2, num_heads=2, context_length=inputs.shape[0])
mha1(batch)

tensor([[[-0.5939, -0.0048, -0.1175,  0.0239],
         [-0.6131, -0.1510, -0.2051,  0.2834],
         [-0.4096, -0.0972, -0.1347,  0.1799],
         [-0.2493, -0.0624, -0.2145,  0.3590],
         [-0.3737, -0.1078, -0.1126,  0.1841],
         [-0.4237, -0.1626, -0.1360,  0.2489]],

        [[ 0.0000,  0.0000, -0.1175,  0.0239],
         [ 0.0000,  0.0000, -0.0561,  0.0114],
         [-0.6125, -0.1987, -0.2326,  0.3586],
         [-0.5532, -0.2132, -0.2145,  0.3590],
         [-0.3737, -0.1078, -0.2130,  0.2978],
         [-0.3865, -0.1549, -0.1497,  0.2781]]], grad_fn=<CatBackward0>)

## Multihead with weight split

In [ ]:
# why we need this? because when we try to implement multihead attention with bunch of causal attn, we do matrix multiplication
# which is compulationally heavy. so we split the weights for key, query and value wt matrix.


In [ ]:
# other thing to note is, no. of feature(col dim of token embedding / head_dim) * no. of heads = d_out (in our wt matrix i.e d_in x d_out)
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [ ]:
import torch
import torch.nn as nn

In [ ]:
class MultiHeadAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length, no_heads, qkv_bias=False, dropout=0.2):
        super().__init__()
        self.d_in = d_in
        self.d_out = d_out
        self.context_length = context_length
        self.no_heads = no_heads
        # we will also need head_dim
        self.head_dim = d_out// no_heads

        # then we initialise weights
        self.W_queries = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_keys = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_values = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )


    def forward(self, x):
        no_of_batches, token_nos, d_in = x.shape

        # initialising key, queries and values
        queries = self.W_queries(x)
        keys = self.W_keys(x)
        values = self.W_values(x)

        # we will view this/split this other way
        queries = queries.view(no_of_batches, token_nos, self.no_heads, self.head_dim)
        keys = keys.view(no_of_batches, token_nos, self.no_heads, self.head_dim)
        values = values.view(no_of_batches, token_nos, self.no_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        queries = queries.transpose(1,2)
        values = values.transpose(1,2)
        keys = keys.transpose(1,2)

        attn_score = queries @ keys.transpose(2,3) # here keys dim could be [1,3,2,3] --> so transpose will be for dim 1,2 --. [1,2,3,3]

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:token_nos, :token_nos]

        attn_score.masked_fill(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_score / keys.shape[-1]**0.5, dim=-1)
        attn_weights= self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(no_of_batches, token_nos, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

## Dummy GPT Architecture

In [ ]:
# text -> tokens -> token_ids -> token_embeddings + position_embeddings -> input_embeddings
# this goes into transformer block.

In [ ]:
GPT_CONFIGURATION_PARAMS = {
    "vocab_size": 50257,
    "n_heads": 12,
    "context_length": 1024,
    "emb_dim": 768,
    "n_layers": 12,
    "dropout": 0.1,
    "qkv_bias": False
}

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class DummyGPTarchitecture(nn.Module):

    def __init__(self, cfg): #cfg - configuration block
        super().__init__()
        self.token_emb = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.position_emb = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.drop_rate = nn.Dropout(cfg['dropout'])

        # initialising transformerb lock
        self.transformer_blocks = nn.Sequential(
        *[DummyTransformerBlock(cfg) for _ in range(cfg['n_layers'])])

        # and then, there was a normalisation layer
        self.final_norm = DummyNormLayer(cfg['emb_dim'])

        #and final output
        self.out_head=nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )


    def forward(self, id_idx):
        batch_size, sequence_len = id_idx.shape
        tok_embs = self.token_emb(id_idx)
        pos_embs = self.position_emb(torch.arange(sequence_len, device=id_idx.device))
        input_embs = tok_embs + pos_embs
        input_embs = self.drop_rate(input_embs)
        input_embs = self.transformer_blocks(input_embs)
        input_embs = self.final_norm(input_embs)
        logits = self.out_head(input_embs)

        return logits


class DummyTransformerBlock(nn.Module):
      def __init__(self,cfg):
          super().__init__()

      def forward(self, x):
          return x

class DummyNormLayer(nn.Module):
      def __init__(self, norm_shape, eps = 1e-5):
          super().__init__()

      def forward(self,x):
          return x






## step 1: Tokenization

In [ ]:
import tiktoken

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
batch=[]
txt1 = "I am learning LLM"
txt2 = "IT is intersting"
batch.append(torch.tensor(tokenizer.encode(txt1)))

In [ ]:
batch.append(torch.tensor(tokenizer.encode(txt2)))

In [ ]:
print(batch)

[tensor([   40,   716,  4673, 27140,    44]), tensor([2043,  318,  987,  301,  278])]


In [ ]:
batch = torch.stack(batch, dim=0)
batch

tensor([[   40,   716,  4673, 27140,    44],
        [ 2043,   318,   987,   301,   278]])

In [ ]:
batch

tensor([[   40,   716,  4673, 27140,    44],
        [ 2043,   318,   987,   301,   278]])

In [ ]:
# creating instance of dummy gpt

In [ ]:
torch.manual_seed(123)
gpt = DummyGPTarchitecture(GPT_CONFIGURATION_PARAMS)
logits = gpt(batch)

In [ ]:
print(logits)

tensor([[[-0.7108,  0.5645, -0.5606,  ...,  1.3645, -0.3677,  0.9244],
         [ 0.8104, -1.1496, -1.6109,  ...,  0.5917, -0.5173,  2.0143],
         [ 0.7115, -0.7746, -0.4637,  ..., -0.3947,  0.7802, -0.8231],
         [ 0.6124,  0.2343, -0.5435,  ..., -0.4446, -1.6082, -0.1409],
         [-1.2540,  0.4096,  0.4182,  ..., -1.7074,  0.2678, -0.8032]],

        [[-0.1162,  0.0534,  0.9246,  ...,  0.8270,  0.3381, -0.3394],
         [-0.4163, -0.9663, -2.6294,  ...,  2.2240, -1.1728,  1.7489],
         [ 1.7931, -0.7639,  0.2189,  ...,  0.1075,  1.7238,  0.6194],
         [ 0.0846, -0.0361, -0.3526,  ..., -0.8309, -0.3208,  2.0221],
         [-0.9283,  0.1035,  0.7228,  ..., -0.8316,  0.8789, -2.0160]]],
       grad_fn=<UnsafeViewBackward0>)


In [ ]:
# even thought this is random, it worked, so i am happy!

## Step 2: Layer Normalization

In [ ]:
# below is example for this

In [ ]:
import torch

In [ ]:
torch.manual_seed(123)
random_sample=torch.randn(3,5)

In [ ]:
import torch.nn as nn

In [ ]:
layer = nn.Sequential(nn.Linear(5,6), nn.ReLU())
output = layer(random_sample)

In [ ]:
output

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.3571, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2610],
        [0.0000, 0.0000, 0.0000, 0.0811, 0.0000, 0.1129]],
       grad_fn=<ReluBackward0>)

In [ ]:
mean = output.mean(dim=-1, keepdim=True)
print(mean)


tensor([[0.0595],
        [0.0435],
        [0.0323]], grad_fn=<MeanBackward1>)


In [ ]:
var = output.var(dim=-1, keepdim=True)
print(var)

tensor([[0.0212],
        [0.0114],
        [0.0026]], grad_fn=<VarBackward0>)


In [ ]:
norm_out =( output - mean)/ torch.sqrt(var)

In [ ]:
print(norm_out.mean(dim=-1,keepdim=True))

tensor([[-9.9341e-09],
        [-9.9341e-09],
        [-9.9341e-09]], grad_fn=<MeanBackward1>)


In [ ]:
torch.set_printoptions(sci_mode=False)

In [ ]:
print(norm_out.mean(dim=-1,keepdim=True))

tensor([[-0.0000],
        [-0.0000],
        [-0.0000]], grad_fn=<MeanBackward1>)


In [ ]:
# now making a layernorm class
class LayerNormalization(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x-mean)/torch.sqrt(var + self.eps)

        return self.scale*norm_x + self.shift


In [ ]:
ln = LayerNormalization(emb_dim=5)
out_ln = ln(random_sample)
mean = out_ln.mean(dim=-1, keepdim=True)
var = out_ln.var(dim=-1, unbiased=False, keepdim=True)
print("Mean:\n", mean)
print("Variance:\n", var)

Mean:
 tensor([[-0.0000],
        [ 0.0000],
        [ 0.0000]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000],
        [0.9999]], grad_fn=<VarBackward0>)


## GELU Activation function and Feedforward NN

In [ ]:
# used in feed forward nn, in transformer block

In [ ]:
import torch.nn as nn
import torch

In [ ]:
class GELUActivation(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        # we use approximation
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [ ]:
class feedForwardNN(nn.Module):
    # this has linear layer, expansion, contraction
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg['emb_dim'], 4*cfg['emb_dim']),
            GELUActivation(),
            nn.Linear(4*cfg['emb_dim'], cfg['emb_dim'])
        )

    def forward(self, x):
        output = self.layers(x)
        return output


In [ ]:
ffn = feedForwardNN(GPT_CONFIGURATION_PARAMS)
x = torch.randn(2,3,768)
a = ffn(x)
print(a.shape)


torch.Size([2, 3, 768])


## Shortcut connections

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class shortcutConnections(nn.Module):
    def __init__(self, layer_size, use_shortcut):
        super().__init__()
        self.use_shortcut = use_shortcut
        self.model = nn.ModuleList([
            nn.Sequential(nn.Linear(layer_size[0], layer_size[1], GELUActivation())),
            nn.Sequential(nn.Linear(layer_size[1], layer_size[2], GELUActivation())),
            nn.Sequential(nn.Linear(layer_size[2], layer_size[3], GELUActivation())),
            # nn.Sequential(nn.Linear(layer_size[3], layer_size[4], GELUActivation()))
        ])

    def forward(self, x):
        # x is input
        for layer in self.model:
            output = layer(x)
            if self.use_shortcut and x.shape == output.shape:
                x = output + x
            else:
                x = output

        return output

In [ ]:
layer_sizes = [3,3,3,1]
inputs = [1. , 0. , -1.]
inputs = torch.tensor(inputs)

In [ ]:
torch.manual_seed(123)
model_without_shortcut = shortcutConnections(layer_sizes, False)

In [ ]:
abc = iter(model_without_shortcut.named_parameters())
next(abc)

NameError: name 'model_without_shortcut' is not defined

In [ ]:
def print_gradients(model, x):
    # Forward pass
    output = model(x)
    target = torch.tensor([[0.]])

    # Calculate loss based on how close the target
    # and output are
    loss = nn.MSELoss()
    loss = loss(output, target)

    # Backward pass to calculate the gradients
    loss.backward()

    for name, param in model.named_parameters():
        if 'weight' in name:
            # Print the mean absolute gradient of the weights
            print(f"{name} has gradient mean of {param.grad.abs().mean().item()}")

In [ ]:
print(print_gradients(model_without_shortcut, inputs))

model.0.0.weight has gradient mean of 0.05872293934226036
model.1.0.weight has gradient mean of 0.09259593486785889
model.2.0.weight has gradient mean of 0.4166198968887329
None


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
model_with_shortcut = shortcutConnections(layer_sizes, True)
print(print_gradients(model_with_shortcut, inputs))

model.0.0.weight has gradient mean of 0.07365316152572632
model.1.0.weight has gradient mean of 0.035481046885252
model.2.0.weight has gradient mean of 0.31351932883262634
None


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


## Transformer Architecture

In [ ]:
class TransformerArchitect(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(cfg["emb_dim"], cfg["emb_dim"], cfg["context_length"], cfg["n_heads"], cfg["qkv_bias"], cfg["dropout"])
        self.dropout_layer = nn.Dropout(cfg["dropout"])
        self.norm1 = LayerNormalization(cfg["emb_dim"])
        self.norm2 = LayerNormalization(cfg["emb_dim"])
        self.feed_forward = feedForwardNN(cfg)

    def forward(self, x):
        og_input = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.dropout_layer(x)

        x = x + og_input

        og_input = x
        x = self.norm2(x)
        x = self.feed_forward(x)
        x = self.dropout_layer(x)
        x = x + og_input

        return x

In [ ]:
GPT_CONFIGURATION_PARAMS = {
    "vocab_size": 50257,
    "n_heads": 12,
    "context_length": 1024,
    "emb_dim": 768,
    "n_layers": 12,
    "dropout": 0.1,
    "qkv_bias": False
}

In [ ]:
torch.manual_seed(123)
x = torch.rand(2,4,768)
transformer = TransformerArchitect(GPT_CONFIGURATION_PARAMS)
y = transformer(x)
print(y.shape)

torch.Size([2, 4, 768])


## GPT Architecture from scratch (124M Parameters)

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class GPTArchitecture(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.embedding_layer = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.pos_emb_layer = nn.Embedding(cfg['context_length'], cfg['emb_dim'])
        self.drop_emb = nn.Dropout(cfg['dropout'])

        self.transformer_blocks = nn.Sequential( *[TransformerArchitect(cfg) for _ in range(cfg['n_layers'])])
        self.final_norm = LayerNormalization(cfg['emb_dim'])
        self.output_head = nn.Linear(
            cfg['emb_dim'], cfg['vocab_size'], bias=False
        )

    def forward(self, id_idx):
        batch_size, seq_len = id_idx.shape
        token_emb = self.embedding_layer(id_idx)
        pos_emb = self.pos_emb_layer(torch.arange(seq_len, device=id_idx.device))
        x = token_emb + pos_emb
        x = self.drop_emb(x)
        x = self.transformer_blocks(x)
        x = self.final_norm(x)
        logits = self.output_head(x)
        return logits






In [ ]:
torch.manual_seed(123)
model = GPTArchitecture(GPT_CONFIGURATION_PARAMS)
# Corrected: Generate integer tensor for token IDs
batch = torch.randint(0, GPT_CONFIGURATION_PARAMS['vocab_size'], (2, 4), dtype=torch.long)
out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[25371, 42188, 47556,  8856],
        [13224, 29264, 30628,  7204]])

Output shape: torch.Size([2, 4, 50257])
tensor([[[-0.4839, -1.2559, -0.3680,  ...,  0.3904,  0.1283, -0.3099],
         [ 0.2628, -0.5918,  0.4738,  ..., -0.8778,  0.2563, -0.2689],
         [ 1.9126, -0.8372, -0.5461,  ...,  0.4070, -0.3218, -0.9181],
         [-0.2324, -0.2113,  0.4055,  ...,  0.4150,  0.2015, -0.5003]],

        [[-0.2270,  0.6004,  0.4399,  ...,  0.2319, -0.0710, -1.0960],
         [ 0.1872, -0.1137,  0.2155,  ...,  0.1230,  0.0596,  0.7125],
         [ 0.7089,  0.0151,  0.3350,  ...,  0.4523, -0.4390,  1.1430],
         [-0.7643, -0.1387,  0.9632,  ...,  0.2284, -0.4543,  0.1508]]],
       grad_fn=<UnsafeViewBackward0>)


In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 163,009,536


## Predicting the next word

In [ ]:
def predict_next_token(model, idx, max_new_tokens, context_size):
    # if size is < context_size then we drop it
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]

        with torch.no_grad():
          logits = model(idx_cond)

        logits = logits[:, -1, :]

        probs = torch.softmax(logits, dim=-1)
        idx_next = torch.argmax(probs, dim=-1, keepdim=True)  # (batch, 1)
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx


In [ ]:
start_context = "Hey there"


In [ ]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
encode = tokenizer.encode(start_context)
encoded_tensor = torch.tensor(encode).unsqueeze(0)

In [ ]:
print(encoded_tensor)

tensor([[10814,   612]])


In [ ]:
model.eval()

GPTArchitecture(
  (embedding_layer): Embedding(50257, 768)
  (pos_emb_layer): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (transformer_blocks): Sequential(
    (0): TransformerArchitect(
      (multihead_attention): MultiHeadAttention(
        (W_queries): Linear(in_features=768, out_features=768, bias=False)
        (W_keys): Linear(in_features=768, out_features=768, bias=False)
        (W_values): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (dropout_layer): Dropout(p=0.1, inplace=False)
      (norm1): LayerNormalization()
      (norm2): LayerNormalization()
      (feed_forward): feedForwardNN(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELUActivation()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
   

In [ ]:
output = predict_next_token(
    model,
    encoded_tensor,
    max_new_tokens=10,
    context_size=GPT_CONFIGURATION_PARAMS["context_length"]
)

In [ ]:
print(output)

tensor([[10814,   612, 16806, 24906, 27018,  7283, 48443, 26012, 16560, 44407,
         34178, 12156]])


In [ ]:
decode_text = tokenizer.decode(output.squeeze(0).tolist())
print(decode_text)

Hey there Applic HER Feature IT!/ Brigade incorporatedheartedly LIKEDep


## Model Loss function

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class GPTArchitecture(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.embedding_layer = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.pos_emb_layer = nn.Embedding(cfg['context_length'], cfg['emb_dim'])
        self.drop_emb = nn.Dropout(cfg['dropout'])

        self.transformer_blocks = nn.Sequential( *[TransformerArchitect(cfg) for _ in range(cfg['n_layers'])])
        self.final_norm = LayerNormalization(cfg['emb_dim'])
        self.output_head = nn.Linear(
            cfg['emb_dim'], cfg['vocab_size'], bias=False
        )

    def forward(self, id_idx):
        batch_size, seq_len = id_idx.shape
        token_emb = self.embedding_layer(id_idx)
        pos_emb = self.pos_emb_layer(torch.arange(seq_len, device=id_idx.device))
        x = token_emb + pos_emb
        x = self.drop_emb(x)
        x = self.transformer_blocks(x)
        x = self.final_norm(x)
        logits = self.output_head(x)
        return logits






In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}


In [ ]:
torch.manual_seed(123)
model = GPTArchitecture(GPT_CONFIG_124M)

In [ ]:
model.eval()

GPTArchitecture(
  (embedding_layer): Embedding(50257, 768)
  (pos_emb_layer): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (transformer_blocks): Sequential(
    (0): TransformerArchitect(
      (multihead_attention): MultiHeadAttention(
        (W_queries): Linear(in_features=768, out_features=768, bias=False)
        (W_keys): Linear(in_features=768, out_features=768, bias=False)
        (W_values): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (dropout_layer): Dropout(p=0.1, inplace=False)
      (norm1): LayerNormalization()
      (norm2): LayerNormalization()
      (feed_forward): feedForwardNN(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELUActivation()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    

In [ ]:
import tiktoken

def text_to_token_id(tokenizer, text):

    token_ids = tokenizer.encode(text)
    return torch.tensor(token_ids).unsqueeze(0)

def token_ids_to_text(tokenizer, token_ids):
    texts = token_ids.squeeze(0)
    return tokenizer.decode(texts.tolist())



In [ ]:
start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
token_ids = predict_next_token(model,
                               text_to_token_id(tokenizer, start_context),
                               max_new_tokens=10,
                               context_size=GPT_CONFIG_124M["context_length"]
                               )

In [ ]:
print(token_ids_to_text(tokenizer, token_ids))

Every effort moves you rentingetic minion mobilized Macicone warrantyuler respirmediated


## Calculating text generation loss: cross-entropy and perplexity

In [ ]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107,  588, 11311]]) #  " really like chocolate"]

In [ ]:
with torch.no_grad():
    logits = model(inputs)

probas = torch.softmax(logits, dim=-1) # Probability of each token in vocabulary
print(probas.shape) # Shape: (batch_size, num_tokens, vocab_size)

torch.Size([2, 3, 50257])


In [ ]:
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

Token IDs:
 tensor([[[36195],
         [16031],
         [42826]],

        [[14212],
         [ 7822],
         [38509]]])


In [ ]:
print(f"Targets batch 1: {token_ids_to_text(tokenizer, targets[0])}")
print(f"Outputs batch 1: {token_ids_to_text(tokenizer, token_ids[0].flatten())}")

Targets batch 1:  effort moves you
Outputs batch 1: lif savesNetflix


## LLM Performance on real dataset

In [ ]:
# reading the file
with open('/content/sample_data/the-verdict.txt', 'r') as f:
    text_data = f.read()

In [ ]:
print(text_data[:99])

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [ ]:
total_characters = len(text_data)
totak_tokens = len(tokenizer.encode(text_data))

In [ ]:
totak_tokens

5145

In [ ]:
# we will use byte-pair encoding
# we will split dataset into train and val and for that, we will use dataLoader
from torch.utils.data import Dataset, DataLoader

class GPT_Dataset(Dataset):

    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids)-max_length, stride):
              input_chunk = token_ids[i: i+max_length]
              target_chunk = token_ids[i+1 : i+1+max_length]
              self.input_ids.append(torch.tensor(input_chunk))
              self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]



In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPT_Dataset(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "dropout": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}


In [ ]:
# creating batches
split_ratio = 0.9
split_token = int(split_ratio * len(text_data))
train_data = text_data[:split_token]
val_data = text_data[split_token:]

train_batch = create_dataloader_v1(
    train_data,
    2,
    GPT_CONFIG_124M["context_length"],
    GPT_CONFIG_124M["context_length"],
    True,
    True,
    num_workers=0
)

val_batch = create_dataloader_v1(
    val_data,
    2,
    GPT_CONFIG_124M["context_length"],
    GPT_CONFIG_124M["context_length"],
    True,
    True,
    num_workers=0
)



In [ ]:
train_tokens = 0
for input_batch, target_batch in train_batch:
    train_tokens += input_batch.numel()

val_tokens = 0
for input_batch, target_batch in val_batch:
    val_tokens += input_batch.numel()

print("Training tokens:", train_tokens)
print("Validation tokens:", val_tokens)
print("All tokens:", train_tokens + val_tokens)

Training tokens: 4608
Validation tokens: 512
All tokens: 5120


In [ ]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = calc_loss_loader(train_batch, model, device)
    val_loss = calc_loss_loader(val_batch, model, device)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

NameError: name 'calc_loss_loader' is not defined

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## LLM Pre-training loop

In [ ]:
# here, we will try to minimize loss function as much as possible
# so, output as close to target

In [ ]:
 # backward pass -> allows to calculate gradient of the loss, which update the model weights and parameters
 # we find loss gradients --> loss.backward()


In [ ]:
def model_training_simple_version(model, train_loader, val_loader, optimizer
                                  , device, num_epochs, eval_freq, eval_iter, start_context, tokenizer):
    train_losses, val_losses, track_tokens_seen = [],[],[]
    tokens_seen, global_step = 0,-1

    for epoch in range(num_epochs):
        model.train()

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad()
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward()
            optimizer.step()
            tokens_seen += input_batch.numel() #total no of elements in input_batch
            global_step += 1


            # evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch
        generate_and_print_sample(
            model, tokenizer, device, start_context
        )

    return train_losses, val_losses, track_tokens_seen

In [ ]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

In [ ]:
def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb_layer.weight.shape[0]
    encoded = text_to_token_id(tokenizer, start_context).to(device)
    with torch.no_grad():
        token_ids = predict_next_token(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
    decoded_text = token_ids_to_text(tokenizer, token_ids)
    print(decoded_text.replace("\n", " "))  # Compact print format
    model.train()

In [ ]:
# Note:
# Uncomment the following code to calculate the execution time
import time
start_time = time.time()

torch.manual_seed(123)
model = GPTArchitecture(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, tokens_seen = model_training_simple_version(
    model, train_batch, val_batch, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer
)

# Note:
# Uncomment the following code to show the execution time
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train loss 9.778, Val loss 9.927
Ep 1 (Step 000005): Train loss 8.077, Val loss 8.337
Every effort moves you,,,,,,,,,,,,,,.                                   
Ep 2 (Step 000010): Train loss 6.757, Val loss 7.041
Ep 2 (Step 000015): Train loss 6.100, Val loss 6.597
Every effort moves you, the,, the,,,,,,,,,.                                   
Ep 3 (Step 000020): Train loss 5.831, Val loss 6.642
Ep 3 (Step 000025): Train loss 5.123, Val loss 6.362
Every effort moves you of theisburn to the picture to the to the pictureburn, and I had to the picture to the picture to have of the picture. I had been the picture to the my to the picture to the pictureburn's of the pictureburn,
Ep 4 (Step 000030): Train loss 5.049, Val loss 6.271
Ep 4 (Step 000035): Train loss 4.704, Val loss 6.240
Every effort moves you know; and my dear was not to have to have to have to have my painting, and I was, and he was not to have of the pictureburn, and he was his pictures, and he was, and he w

## Model with Temperature and Top_K Sampling

In [ ]:
def softmax_with_temp(logits, temp):
    scaled_logits = logits / temp
    return torch.softmax(scaled_logits, dim=0)


In [ ]:
  # torch.topk(logits, top_k_number) -> returns top k logits, and their positions

In [ ]:
import torch

In [ ]:
def generate(model, idx, max_new_tokens, context_size, temperature=1.0, top_k=None, eos_id=None):
    for _ in range(max_new_tokens):
        # Crop idx to the last context_size tokens to respect context window
        idxs_cond = idx[:, -context_size:]

        with torch.no_grad():
            logits = model(idxs_cond)

        # Focus on the last token's logits
        logits = logits[:, -1, :]

        # Apply top-k filtering
        if top_k is not None:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1))) # Ensure k is not larger than vocab size
            logits[logits < v[:, [-1]]] = float('-inf') # Set smaller logits to -inf

        # Apply temperature scaling
        if temperature != 1.0:
            logits = logits / temperature

        probas = torch.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probas, num_samples=1) # Sample next token

        # Check for end-of-sequence token
        if eos_id is not None and (idx_next == eos_id).all():
            break

        idx = torch.cat((idx, idx_next), dim=1) # Append the sampled token

    return idx

# Downloading OpenAI Weights

In [ ]:
import os
import requests  # Make sure requests is installed
import json
import numpy as np
import tensorflow as tf
from tqdm import tqdm

In [ ]:
def download_and_load_gpt2(model_size, models_dir):
    # Validate model size
    allowed_sizes = ("124M", "355M", "774M", "1558M")
    if model_size not in allowed_sizes:
        raise ValueError(f"Model size not in {allowed_sizes}")

    # Define paths
    model_dir = os.path.join(models_dir, model_size)
    base_url = "https://openaipublic.blob.core.windows.net/gpt-2/models"
    filenames = [
        "checkpoint", "encoder.json", "hparams.json",
        "model.ckpt.data-00000-of-00001", "model.ckpt.index",
        "model.ckpt.meta", "vocab.bpe"
    ]

    # Download files
    os.makedirs(model_dir, exist_ok=True)
    for filename in filenames:
        file_url = os.path.join(base_url, model_size, filename)
        file_path = os.path.join(model_dir, filename)
        download_file(file_url, file_path)

    ## We have reached here until now ---> we have downloaded the files on our local machine.

    # Load settings and params
    tf_ckpt_path = tf.train.latest_checkpoint(model_dir)
    settings = json.load(open(os.path.join(model_dir, "hparams.json")))
    params = load_gpt2_params_from_tf_ckpt(tf_ckpt_path, settings)

    return settings, params

def download_file(url, destination):
    try:
        # Send a GET request to download the file, disabling SSL verification
        response = requests.get(url, stream=True, verify=False)

        # Get the total file size from headers, defaulting to 0 if not present
        file_size = int(response.headers.get("content-length", 0))

        # Check if file exists and has the same size
        if os.path.exists(destination):
            file_size_local = os.path.getsize(destination)
            if file_size == file_size_local:
                print(f"File already exists and is up-to-date: {destination}")
                return

        # Define the block size for reading the file
        block_size = 1024  # 1 Kilobyte

        # Initialize the progress bar with total file size
        progress_bar_description = url.split("/")[-1]  # Extract filename from URL
        with tqdm(total=file_size, unit="iB", unit_scale=True, desc=progress_bar_description) as progress_bar:
            # Open the destination file in binary write mode
            with open(destination, "wb") as file:
                # Iterate over the file data in chunks
                for chunk in response.iter_content(block_size):
                    progress_bar.update(len(chunk))  # Update progress bar
                    file.write(chunk)  # Write the chunk to the file

    except requests.exceptions.RequestException as e:
        print(f"Error downloading the file: {e}")
        print(f"Please check the URL: {url}")

def load_gpt2_params_from_tf_ckpt(ckpt_path, settings):
    # Initialize parameters dictionary with empty blocks for each layer
    params = {"blocks": [{} for _ in range(settings["n_layer"])]}

    # Iterate over each variable in the checkpoint
    for name, _ in tf.train.list_variables(ckpt_path):
        # Load the variable and remove singleton dimensions
        variable_array = np.squeeze(tf.train.load_variable(ckpt_path, name))

        # Process the variable name to extract relevant parts
        variable_name_parts = name.split("/")[1:]  # Skip the 'model/' prefix

        # Identify the target dictionary for the variable
        target_dict = params
        if variable_name_parts[0].startswith("h"):
            layer_number = int(variable_name_parts[0][1:])
            target_dict = params["blocks"][layer_number]

        # Recursively access or create nested dictionaries
        for key in variable_name_parts[1:-1]:
            target_dict = target_dict.setdefault(key, {})

        # Assign the variable array to the last key
        last_key = variable_name_parts[-1]
        target_dict[last_key] = variable_array

    return params

In [ ]:
settings, params = download_and_load_gpt2(model_size="124M", models_dir="/content/gpt2")

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
checkpoint: 100%|██████████| 77.0/77.0 [00:00<00:00, 114kiB/s]
/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
encoder.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 2.93MiB/s]
/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verificat

In [ ]:
print("Settings", settings)

Settings {'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12}


In [ ]:
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

# Copy the base configuration and update with specific model settings
model_name = "gpt2-small (124M)"  # Example model name
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])

In [ ]:
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})
gpt = GPTModel(NEW_CONFIG)
gpt.eval();

In [ ]:
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch. Left: {left.shape}, Right: {right.shape}")
    return torch.nn.Parameter(torch.tensor(right))

In [ ]:
# @title
import numpy as np

def load_weights_into_gpt(gpt, params):
    gpt.pos_emb.weight = assign(gpt.pos_emb.weight, params['wpe'])
    gpt.tok_emb.weight = assign(gpt.tok_emb.weight, params['wte'])

    for b in range(len(params["blocks"])):
        q_w, k_w, v_w = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.weight = assign(
            gpt.trf_blocks[b].att.W_query.weight, q_w.T)
        gpt.trf_blocks[b].att.W_key.weight = assign(
            gpt.trf_blocks[b].att.W_key.weight, k_w.T)
        gpt.trf_blocks[b].att.W_value.weight = assign(
            gpt.trf_blocks[b].att.W_value.weight, v_w.T)

        q_b, k_b, v_b = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.bias = assign(
            gpt.trf_blocks[b].att.W_query.bias, q_b)
        gpt.trf_blocks[b].att.W_key.bias = assign(
            gpt.trf_blocks[b].att.W_key.bias, k_b)
        gpt.trf_blocks[b].att.W_value.bias = assign(
            gpt.trf_blocks[b].att.W_value.bias, v_b)

        gpt.trf_blocks[b].att.out_proj.weight = assign(
            gpt.trf_blocks[b].att.out_proj.weight,
            params["blocks"][b]["attn"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].att.out_proj.bias = assign(
            gpt.trf_blocks[b].att.out_proj.bias,
            params["blocks"][b]["attn"]["c_proj"]["b"])

        gpt.trf_blocks[b].ff.layers[0].weight = assign(
            gpt.trf_blocks[b].ff.layers[0].weight,
            params["blocks"][b]["mlp"]["c_fc"]["w"].T)
        gpt.trf_blocks[b].ff.layers[0].bias = assign(
            gpt.trf_blocks[b].ff.layers[0].bias,
            params["blocks"][b]["mlp"]["c_fc"]["b"])
        gpt.trf_blocks[b].ff.layers[2].weight = assign(
            gpt.trf_blocks[b].ff.layers[2].weight,
            params["blocks"][b]["mlp"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].ff.layers[2].bias = assign(
            gpt.trf_blocks[b].ff.layers[2].bias,
            params["blocks"][b]["mlp"]["c_proj"]["b"])

        gpt.trf_blocks[b].norm1.scale = assign(
            gpt.trf_blocks[b].norm1.scale,
            params["blocks"][b]["ln_1"]["g"])
        gpt.trf_blocks[b].norm1.shift = assign(
            gpt.trf_blocks[b].norm1.shift,
            params["blocks"][b]["ln_1"]["b"])
        gpt.trf_blocks[b].norm2.scale = assign(
            gpt.trf_blocks[b].norm2.scale,
            params["blocks"][b]["ln_2"]["g"])
        gpt.trf_blocks[b].norm2.shift = assign(
            gpt.trf_blocks[b].norm2.shift,
            params["blocks"][b]["ln_2"]["b"])

    gpt.final_norm.scale = assign(gpt.final_norm.scale, params["g"])
    gpt.final_norm.shift = assign(gpt.final_norm.shift, params["b"])
    gpt.out_head.weight = assign(gpt.out_head.weight, params["wte"])

In [ ]:
load_weights_into_gpt(gpt, params)
gpt.to(device);

In [ ]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNormalization(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = feedForwardNN(cfg)
        self.norm1 = LayerNormalization(cfg["emb_dim"])
        self.norm2 = LayerNormalization(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        # 2*4*768
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [ ]:
class TransformerArchitect(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(cfg["emb_dim"], cfg["emb_dim"], cfg["context_length"], cfg["n_heads"], cfg["qkv_bias"], cfg["dropout"])
        self.dropout_layer = nn.Dropout(cfg["dropout"])
        self.norm1 = LayerNormalization(cfg["emb_dim"])
        self.norm2 = LayerNormalization(cfg["emb_dim"])
        self.feed_forward = feedForwardNN(cfg)

    def forward(self, x):
        og_input = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.dropout_layer(x)

        x = x + og_input

        og_input = x
        x = self.norm2(x)
        x = self.feed_forward(x)
        x = self.dropout_layer(x)
        x = x + og_input

        return x

In [ ]:
class GPTArchitecture(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.embedding_layer = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.pos_emb_layer = nn.Embedding(cfg['context_length'], cfg['emb_dim'])
        self.drop_emb = nn.Dropout(cfg['dropout'])

        self.transformer_blocks = nn.Sequential( *[TransformerArchitect(cfg) for _ in range(cfg['n_layers'])])
        self.final_norm = LayerNormalization(cfg['emb_dim'])
        self.output_head = nn.Linear(
            cfg['emb_dim'], cfg['vocab_size'], bias=False
        )

    def forward(self, id_idx):
        batch_size, seq_len = id_idx.shape
        token_emb = self.embedding_layer(id_idx)
        pos_emb = self.pos_emb_layer(torch.arange(seq_len, device=id_idx.device))
        x = token_emb + pos_emb
        x = self.drop_emb(x)
        x = self.transformer_blocks(x)
        x = self.final_norm(x)
        logits = self.output_head(x)
        return logits






In [ ]:
import tiktoken

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
torch.manual_seed(123)

token_ids = generate(
    model=gpt,
    idx=text_to_token_id(tokenizer, "always thought Jack Gisburn").to(device),
    max_new_tokens=25,
    context_size=NEW_CONFIG["context_length"],
    top_k=50,
    temperature=1.5
)

In [ ]:
print("Output text:\n", token_ids_to_text(tokenizer, token_ids))

Output text:
 always thought Jack Gisburn, Jr. would make a great quarterback at QB this season. The Bills did not come on, but Gisburn would


In [ ]:
discipline_quotes = generate(
    model=gpt,
    idx=text_to_token_id(tokenizer, "Early to bed early to rise").to(device),
    max_new_tokens=30,
    context_size=NEW_CONFIG["context_length"],
    top_k=50,
    temperature=0.8
)

print("Output text:\n", token_ids_to_text(tokenizer, discipline_quotes))

Output text:
 Early to bed early to rise. It was almost always the same morning. It was very cold. The weather was always very bad. Our house was very cold. My husband wasn


In [ ]:
print("hi")

hi


In [ ]:
discipline_quotes = generate(
    model=gpt,
    idx=text_to_token_id(tokenizer, "In a world where quantum kangaroos exist, they jump over particles and").to(device),
    max_new_tokens=30,
    context_size=NEW_CONFIG["context_length"],
    top_k=50,
    temperature=0.8
)

print("Output text:\n", token_ids_to_text(tokenizer, discipline_quotes))

Output text:
 In a world where quantum kangaroos exist, they jump over particles and cause waves in the real world so that they can be made to move.

This is why quantum kangaroos have such profound properties.


In [ ]:
discipline_quotes = generate(
    model=gpt,
    idx=text_to_token_id(tokenizer, "In a world where calendars breathe, people learn to").to(device),
    max_new_tokens=30,
    context_size=NEW_CONFIG["context_length"],
    top_k=50,
    temperature=0.8
)

print("Output text:\n", token_ids_to_text(tokenizer, discipline_quotes))

Output text:
 In a world where calendars breathe, people learn to take things in stride. It's a whole different story than we all know.

"I think people should be interested in making the choices they


In [ ]:
torch.save(gpt.state_dict(), "/content/gpt_model.pt")

In [ ]:
# Save the whole model (architecture + weights)
torch.save(gpt, "/content/gpt2_124M_full_model.pt")
